# Исследование надежности клиентов, подавших заявку на кредит

# Описание проекта

- Заказчик проекта — отдел банка, отвечающий за выдачу кредитов. 

- Задача проекта — выяснить, как влияет семейное положение и количество детей заемщика на возврат кредита в установленные сроки. На результатах исследования будет основана модель кредитного скоринга, которая позволит оценить, сможет ли возможный клиент погасить выданный кредит банку.

# Описание данных

Исследование основано на внутренней статистике банка о платежеспособности клиентов. 

Данные выданы в csv-файле. Датасет включает следующие поля:

- children — количество детей
- days_employed — суммарный трудовой стаж, в днях
- dob_years — возраст заемщмка, в годах
- education — образование клиента
- education_id — уникальный идентификатор уровня образования
- family_status — семейное положение
- family_status_id — уникальный идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — наличие задолженности по возврату кредитов
- total_income — суммарный ежемесячный доход
- purpose — цель получения кредита

# Содержание проекта

1. [Шаг 1. Загрузка датасета и подготовка его к анализу](#start)
    * [Выводы:](#start_total)
2. [Шаг 2. Предобработка данных.](#preprocessing)
    * [Обработка пропусков](#null)
        * [Обработка пропусков. Выводы:](#null_total)
    * [Замена типа данных](#float_to_int)
        * [Замена типа данных. Выводы:](#float_to_int_total)
    * [Обработка дубликатов](#duplicates)
        * [Обработка дубликатов. Выводы:](#duplicates_total)
    * [Лемматизация](#lemmas)
        * [Лемматизация. Выводы:](#lemmas_total)
    * [Категоризация данных](#category)
        * [Категоризация данных. Выводы:](#category_total)
3. [Шаг 3. Ответы на вопросы](#answers)
4. [Шаг 4. Общий вывод](#total)

### Шаг 1. Загрузка датасета и подготовка его к анализу<a id="start"></a>

In [1]:
# <импорт библиотеки pandas>
import pandas as pd

In [2]:
research = pd.read_csv('...')
research.head()

In [4]:
# <получение общей информации о данных в таблице>
research.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Выводы: <a id="start_total"></a>

Таблица состоит из 21525 строк с дубликатами. 
Каждая строка таблицы содержит информацию о клиенте, подающем заявку на кредит. 

Основные проблемы, которые нужно решать: пропуски в столбцах "days_employed" и "total_income", отрицательные числа, для которых которые нужно взять модуль, слишком большие показатели в столбце "days_employed", дубликаты в столбце education.

Для ответов на вопросы необходимы столбцы children, family_status, total_income и purpose - на них будет сделан основной упор в проекте. 

### Шаг 2. Предобработка данных. <a id="preprocessing"></a>

### Обработка пропусков <a id="null"></a>

#### Проверка на наличие пробелов в названии столбцов:

In [5]:
# <перечень названий столбцов таблицы df> 
research.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Пробелов в названии столбцов не обнаружено

#### Проверка на наличие пропусков:

In [6]:
# <суммарное количество пропусков, выявленных методом isnull() в таблице>
research.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Обнаружены пропуски (2174 штук) в столбцах days_employed и total_income

#### Проверка на наличие аномально больших или отрицательных значений:

In [7]:
# < максимум и минимум столбца "days_employed">
print('Максимальный общий трудовой стаж: ', research['days_employed'].max())
print('Минимальный общий трудовой стаж: ', research['days_employed'].min())
print()

# < максимум и минимум столбца "total_income">
print('Максимальный ежемесячный доход: ', research['total_income'].max())
print('Минимальный ежемесячный доход: ', research['total_income'].min())

Максимальный общий трудовой стаж:  401755.40047533
Минимальный общий трудовой стаж:  -18388.949900568383

Максимальный ежемесячный доход:  2265604.028722744
Минимальный ежемесячный доход:  20667.26379327158


Обнаружены следующие проблемы:
1. Слишком большой трудовой стаж, который может повлиять на среднее значение столбца, поэтому берем вместо среднего - медиану
2. Отрицательный трудовой стаж, для готорого необходимо взять модуль
3. Пропуски в столбце total_income зависят от income_type, необходим цикл

#### Заполнение пропусков столбца "days_employed"

In [8]:
# < Берем значения по модулю, так как отрицательного стажа быть не может >
research['days_employed'] = research['days_employed'].abs()

# < Заполняем пропущенные значения на медианное значение, так как этот столбец нам не нужен для ответов на вопросы >
research['days_employed'] = research['days_employed'].fillna(value = research['days_employed'].median())

print(research['days_employed'].median())

# Проверка наличия пропусков
research.info()


2194.220566878695
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В столбце days_employed больше нет пропусков.
Заполнили пропуски медианным значением, так как этот столбец не нужен для ответов на вопросы.

#### Заполнение пропусков столбца "total_income"

In [9]:
# < Группируем клиентов по типу дохода и выводим среднее значение по каждому типу дохода >
# < Медиану можем не использовать, так как нет аномально больших значений по данному столбцу >
research_groupby_income_type = research.groupby(['income_type'])['total_income'].mean()

def income_type_func(row):
    income_type = row['income_type']
    total_income = row['total_income']
    
# <Создаем условие: если в столбце total_income будет значение NAN, то в него вводится среднее значение, 
# которое посчитано для конкретного типа дохода в research_groupby_income_type >
    if pd.isna(total_income):
        row['total_income'] = research_groupby_income_type.loc[income_type]
    return row

# < Применение метода apply() и параметра axis: со значением 1, чтобы применить метод ко всем строкам датафрейма >
research = research.apply(income_type_func, axis = 1)

# Проверка наличия пропусков
research.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В столбце "total_income" больше нет пропусков

#### Итоговая проверка пропусков в таблице

In [10]:
# <итоговая проверка наличия пропусков в таблице>
research.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Обработка пропусков. Выводы: <a id="null_total"></a>

В этой части проекта выявили пропуски в двух столбцах: days_employed и total_income:

1. Пропуски в days_employed заменили на медианное значение столбца.
2. Пропуски в total_income зависят от типа дохода income_type, поэтому составили цикл, по которому вместо значения NAN взяли среднее значение каждого из типов дохода.

Заключительная проверка показала отсутствие пропусков в таблице research.

Причины появления пропусков в этих столбцах:
1. Возможной причиной появления пропусков в столбце days_employed является то, что у человека нет стажа работы.
2. Возможной причиной появления пропусков в столбце total_income является то, что человек на данный момент не устроен на работу и не получает доход.

### Замена типа данных <a id="float_to_int"></a>

In [11]:
# < Цикл для перевода в int c помощью метода astype>
# < Приводим столбец total_income к целочисленному типу для увеличения точности (копейки будут мешать при подсчете) >
# < Поскольку столбец days_employed не нужен для ответов на вопросы, также приводим его к целочисленному значению, а не формату даты >

for column in ['children', 'days_employed','dob_years', 'education_id', 'family_status_id', 'debt', 'total_income',]:
    research[column] = research[column].astype(int)
    
research.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Замена типа данных. Выводы: <a id="float_to_int_total"></a>

- Заменили тип данных float на int во всех числовых столбцах с помощью цикла, однако на деле - только в столбцах days_employed и total_income.
- Использовали метод astype(int) для перевода значений всего столбца в целочисленный тип данных, не добавляя новый столбец.

### Обработка дубликатов <a id="duplicates"></a>

In [12]:
# <получение суммарного количества дубликатов в таблице >
research.duplicated(keep = False).sum()

106

In [13]:
# <удаление всех дубликатов из таблицы >
research = research.drop_duplicates().reset_index(drop = True)

In [14]:
# <проверка на отсутствие дубликатов>
research.duplicated().sum()

0

In [15]:
# < Количество уникальных значений столбца 'children' >
research['children'].value_counts()

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Необходимо взять под модуль отрицательные значения и исправить выброс/техническую ошибку в 20 детей у 76 человек, поскольку этот показатель влияет на ответы на вопросы исследования.

In [16]:
# < Количество уникальных значений столбца 'children' без отрицательных значений и без технических ошибок в 20 детей>
research['children'] = research['children'].abs()
research['children'] = research['children'].replace(20, 0)
research['children'].value_counts()

0    14183
1     4856
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

Дубликатов строк не обнаружено, исправили техническую ошибку в 20 детей у 76 человек на 0, так как это наиболее часто встречающееся значение и критически это изменение не повлияет на результаты исследования.

In [17]:
# < Количество уникальных значений столбца 'days_employed' >
research['days_employed'].value_counts()

2194    2126
327       16
133       16
438       15
204       14
        ... 
3256       1
5307       1
9405       1
3272       1
2049       1
Name: days_employed, Length: 9086, dtype: int64

Нашли 2126 одинаковых значений "2194" - объясняется тем, что мы заполняли пропуски в этом столбце таблицы медианным значением, данные дубликаты не трогаем.

In [18]:
# < Количество уникальных значений столбца 'dob_years' >
research['dob_years'].value_counts()

35    616
40    607
41    606
34    601
38    597
42    596
33    581
39    572
31    559
36    554
44    545
29    544
30    538
48    537
37    536
50    513
43    512
32    509
49    508
28    503
45    497
27    493
56    484
52    484
47    477
54    476
46    473
53    459
57    456
58    456
51    448
55    443
59    443
26    408
60    374
25    357
61    354
62    349
63    269
24    264
64    262
23    253
65    194
22    183
66    182
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Обнаружено 101 нулевое значение в столбце "dob_years". Заменяем на среднее нулевые значения данного столбца.

In [19]:
# < Среднее значение 'dob_years' и замена 0 на него>
print(research['dob_years'].mean().astype(int))
research['dob_years'] = research['dob_years'].replace(0, 43)
research['dob_years'].value_counts()

43


35    616
43    613
40    607
41    606
34    601
38    597
42    596
33    581
39    572
31    559
36    554
44    545
29    544
30    538
48    537
37    536
50    513
32    509
49    508
28    503
45    497
27    493
52    484
56    484
47    477
54    476
46    473
53    459
57    456
58    456
51    448
55    443
59    443
26    408
60    374
25    357
61    354
62    349
63    269
24    264
64    262
23    253
65    194
22    183
66    182
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Избавились от нулевых значений в столбце dob_years, приведя их к среднему значению

In [20]:
# < Количество уникальных значений столбца 'education' >
research['education'].value_counts()

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Найдены дубликаты, выполняем обработку дубликатов вручную - приводим все значения к нижнему регистру

In [21]:
# < Приведение значений  столбца 'education' к нижнему регистру >
research['education'] = research['education'].str.lower()
research['education'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Привели значения столбца 'education' к нижнему регистру

In [22]:
# < Количество уникальных значений столбца 'education_id' >
research['education_id'].value_counts()

1    15188
0     5251
2      744
3      282
4        6
Name: education_id, dtype: int64

Аномальных значений не обнаружено, ошибок в сопоставлении education и education_id не выявлено

In [23]:
# < Количество уникальных значений столбца 'family_status' >
research['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

Аномальных значений не обнаружено

In [24]:
# < Количество уникальных значений столбца 'family_status_id' >
research['family_status_id'].value_counts()

0    12344
1     4163
4     2810
3     1195
2      959
Name: family_status_id, dtype: int64

Аномальных значений не обнаружено, ошибок в сопоставлении family_status и family_status_id не выявлено

In [25]:
# < Количество уникальных значений столбца 'gender' >
research['gender'].value_counts()

F      14189
M       7281
XNA        1
Name: gender, dtype: int64

Обнаружен выброс в определении пола XNA. Поскольку значение всего одно, заменяем на более часто встречающийся пол F.

In [26]:
# < Количество уникальных значений столбца 'gender' при замене XNA на F>
research['gender'] = research['gender'].replace('XNA', 'F')
research['gender'].value_counts()

F    14190
M     7281
Name: gender, dtype: int64

Исправили выброс - больше варианта XNA в столбце нет

In [27]:
# < Количество уникальных значений столбца 'income_type' >
research['income_type'].value_counts()

сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

Необычных значений не обнаружено

In [28]:
# < Количество уникальных значений столбца 'debt' >
research['debt'].value_counts()

0    19730
1     1741
Name: debt, dtype: int64

Долг либо есть, либо нет - третьего не дано:)

In [29]:
# < Количество уникальных значений столбца 'total_income' >
research['total_income'].value_counts()

161380    1078
202417     503
137127     394
170898     145
177986       3
          ... 
101387       1
138249       1
280240       1
390148       1
264193       1
Name: total_income, Length: 18609, dtype: int64

Обнаружены 4 часто повторяющихся значения месячного дохода:
1. 161380
2. 202417
3. 137127
4. 170898

Это связано с тем, что мы заполняли пропуски в столбце таблице средними значениями, соотносящимися с типом дохода. Данные дубликаты не обрабатываем.

In [30]:
# < Количество уникальных значений столбца 'purpose' >
research['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
операции с жильем                         652
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Обнаружены повторяющиеся значения, для которых необходимо провести лемматизацию

In [31]:
# < Итоговая проверка уникальных значений в столбцах >
for column in ['children', 'education', 'education_id', 'family_status', 'family_status_id', 'gender', 'purpose' ]:
    print('Уникальные значения', column, ': ', research[column].unique())
    print()

Уникальные значения children :  [1 0 3 2 4 5]

Уникальные значения education :  ['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']

Уникальные значения education_id :  [0 1 2 3 4]

Уникальные значения family_status :  ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']

Уникальные значения family_status_id :  [0 1 2 3 4]

Уникальные значения gender :  ['F' 'M']

Уникальные значения purpose :  ['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'а

### Обработка дубликатов. Выводы: <a id="duplicates_total"></a>

Были удалены полностью повторяющиеся строки таблицы. Проверено соответствие education & education_id и family_status & family_status_id.

Найдены и исправлены следующие ошибки в данных:

1. Отрицательные значения в столбце children
2. Техническая ошибка в 20 детей у 76 человек
3. 100 нулевых значений в столбце dob_years
4. Дубликаты в столбце education
5. Выброс XNA в столбце gender

Также вывлена необходимость лемматизации в столбце purpose

Скорее всего выбросы и дубликаты в столбцах были получены вследствие ручного набора данных сотрудниками, а не выбора значений из выпадающих списков.


### Лемматизация <a id="lemmas"></a>

In [32]:
# <Импотрируем необходимые для лемматизации библиотеки>

from pymystem3 import Mystem
m = Mystem()

In [33]:
# < Лемматизируем и добавляем полученные значения в новый столбец таблицы - purpose_lemma >
research['purpose_lemma'] = research['purpose'].apply(m.lemmatize)

# < Функция, которая проводит категоризацию (не используем цикл для ускорения работы) >
def purpose_category(lemmas):
    if ('жилье' in lemmas) or ('недвижимость' in lemmas):
        return 'Займ на недвижимость'
    if 'образование' in lemmas:
        return 'Займ на образование'
    if 'автомобиль' in lemmas:
        return 'Займ на автомобиль'
    if 'свадьба' in lemmas:
        return 'Займ на свадьбу'

# < Добавляем полученные значения в purpose >
research['category'] = research['purpose_lemma'].apply(purpose_category)


# < Удаляем столбец purpose_lemma > - если не нужен 
#del research['purpose_lemma']

research.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",Займ на недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",Займ на автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",Займ на недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",Займ на образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",Займ на свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",Займ на недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",Займ на недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",Займ на образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",Займ на свадьбу
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",Займ на недвижимость


In [34]:
# < Уникальные значения по целям кредита >
research['category'].value_counts()

Займ на недвижимость    10814
Займ на автомобиль       4308
Займ на образование      4014
Займ на свадьбу          2335
Name: category, dtype: int64

### Лемматизация. Выводы: <a id="lemmas_total"></a>

Была выполнена лемматизация по столбцу purpose. 

Были выделены 4 основные категории по целям кредита: 

1. Займ на недвижимость  - 10814
2. Займ на автомобиль    -  4308
3. Займ на образование   -  4014
4. Займ на свадьбу       -  2335


Наиболее популярной целью кредита является покупка недвижимости, больше половины кредитов берутся для этой цели. Примерно одинаковое количество кредитов берутся на покупку или замену автомобиля и на образование. Меньше всего клиенты берут кредиты на свадьбу.


### Категоризация данных <a id="category"></a>

Для ответов на вопросы необходимы столбцы children, family_status, total_income, purpose и debt, соответственно категоризацию выполняем именно по ним.

#### Категоризация данных по столбцу children

In [35]:
# < Создаем таблицу с категоризацией по наличию детей >

def children_or_free(children):
    if children == 0:
        return 'нет детей'
    if children >= 1 :
        return 'есть дети'

research['child_category'] = research['children'].apply(children_or_free)

children_dict = research[['children','child_category']]
children_dict['child_category'].value_counts()

нет детей    14183
есть дети     7288
Name: child_category, dtype: int64

#### Категоризация данных по столбцу family_status

In [36]:
# < Создаем таблицу с категоризацией по семейному положению >

family_status_dict = research[['family_status_id','family_status']]
family_status_dict['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

#### Категоризация данных по столбцу total_income

In [37]:
# < Создаем таблицу с категоризацией по уровню месячного дохода >

research['total_income'].quantile([0.25, 0.5, 0.75])

def total_income_count(total_income):
    if total_income <= 107654.5:
        return 'низкий уровень дохода'
    if total_income > 107654.5 and total_income <= 151898.0:
        return 'средний уровень дохода'
    if total_income > 151898.0 and total_income <= 202417.0:
        return 'уровень дохода выше среднего'
    else:
        return 'высокий уровень дохода'

research['total_income_category'] = research['total_income'].apply(total_income_count)
total_income_dict = research[['total_income','total_income_category']]
total_income_dict['total_income_category'].value_counts()

уровень дохода выше среднего    5825
средний уровень дохода          5369
низкий уровень дохода           5368
высокий уровень дохода          4909
Name: total_income_category, dtype: int64

#### Категоризация данных по столбцу purpose выполнена в столбце category

In [38]:
# < Уникальные значения по целям кредита >
research['category'].value_counts()

Займ на недвижимость    10814
Займ на автомобиль       4308
Займ на образование      4014
Займ на свадьбу          2335
Name: category, dtype: int64

### Категоризация данных. Выводы: <a id="category_total"></a>

1. 2/3 клиентов, запрашивающих кредит, не имеют детей.
2. Больше половины клиентов женаты или замужем.
3. По уровню дохода количество клиентов приблизительно одинаковое - кредиты запрашивают как клиенты с низкими доходами, так и клиенты с наиболее высокими доходами.
4. Почти половина клиентов запрашивает кредит для покупки недвижимости.
5. Приблизительно одинаковое количество клиентов запрашивают кредит для покупки автомобиля или на образование.
6. Наименее популярным запросом является кредит на проведение свадьбы.

### Шаг 3. Ответы на вопросы <a id="answers"></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [39]:
# <Сводная таблица с долей должников наглядно демонстрирует зависимость возврата кредита от наличия детей>

children_debt = research.pivot_table(index = 'child_category', columns = 'debt', values = 'dob_years', aggfunc = 'count')
children_debt.columns = ['Нет долга', 'Есть долг']
children_debt['Доля должников'] = (children_debt['Есть долг']/
                                   (children_debt['Есть долг'] + children_debt['Нет долга'])).map('{:.1%}'.format)
children_debt.head()

,Нет долга,Есть долг,Доля должников
child_category,,,
есть дети,6618,670,9.2%
нет детей,13112,1071,7.6%


### Выводы 

Да, есть зависимость между наличием детей у клиентов и возвратом клиентами долга.

Доля должников среди клиентов, у которых есть дети, выше на 1.6%, чем у тех, кто не имеет детей. Возможно, это связано с дополнительными тратами на содержание детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [40]:
# <Сводная таблица с долей должников наглядно демонстрирует зависимость возврата кредита от семейного положения>

family_status_debt = research.pivot_table(index = 'family_status', columns = 'debt', values = 'dob_years', aggfunc = 'count')
family_status_debt.columns = ['Нет долга', 'Есть долг']
family_status_debt['Доля должников'] = (family_status_debt['Есть долг']
                                        /(family_status_debt['Есть долг'] 
                                          + family_status_debt['Нет долга'])).map('{:.1%}'.format)
family_status_debt.head()

,Нет долга,Есть долг,Доля должников
family_status,,,
Не женат / не замужем,2536,274,9.8%
в разводе,1110,85,7.1%
вдовец / вдова,896,63,6.6%
гражданский брак,3775,388,9.3%
женат / замужем,11413,931,7.5%


### Выводы

Да, есть зависимость между семейным положением и возвратом кредита в срок.

1. Наиболее подвержены накоплению долгов неженатые/незамужные клиенты - 9,8% из них не уплатили кредит в срок.
2. Меньше всего (7,1% и 6,6% и 7,5%) долгов имеют клиенты в разводе, вдовцы/вдовы и женатые/замужние. Возможно, это связано с тем, что данные клиенты уже были женаты/замужем и вели семейный бюджет.
3. Показатели клиентов, состоящих в гражданском браке, по невозврату долгов почти такие же высокие, как у незамужних/неженатых (9,3%). Скорее всего это связано с тем, что в официальном браке семейная пара несет равную ответственность перед кредитором, а в гражданском браке каждый выплачивает сам за себя.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [41]:
# <Сводная таблица с долей должников наглядно демонстрирует зависимость возврата кредита от уровня дохода клиента>

total_income_debt = research.pivot_table(index = 'total_income_category', columns = 'debt', values = 'dob_years', aggfunc = 'count')
total_income_debt.columns = ['Нет долга', 'Есть долг']
total_income_debt['Доля должников'] = (total_income_debt['Есть долг']
                                        /(total_income_debt['Есть долг'] 
                                          + total_income_debt['Нет долга'])).map('{:.1%}'.format)
total_income_debt.head()

,Нет долга,Есть долг,Доля должников
total_income_category,,,
высокий уровень дохода,4564,345,7.0%
низкий уровень дохода,4941,427,8.0%
средний уровень дохода,4894,475,8.8%
уровень дохода выше среднего,5331,494,8.5%


### Выводы

Да, есть зависимость между уровнем дохода и возвратом кредита в срок.

1. Наименее подвержены накоплению долгов клиенты с высоким уровнем дохода (7,0%). Сокрее всего это связано с наличием средств для выплаты кредита в срок)
2. Больше всего должников среди клиентов со средним уровнем дохода и уровнем дохода выше среднего - (8,8% и 8,5%). Возможно, это связано с тем, что за последние 15 лет зафиксировано сокращение среднего класса до минимума. То есть те, кто ранее мог себе позволить больше, берут кредиты для больших покупок и не могут их вернуть в срок. (https://www.rbc.ru/economics/24/06/2019/5d10b2d29a79474be6a808ad)
3. Клиенты с низким уровнем дохода (8,0%) ближе всего к среднему значению по доле должников по уровню дохода (8,075). Они обладают средней приверженностью к накоплению долгов.

- Как разные цели кредита влияют на его возврат в срок?

In [42]:
# <Сводная таблица с долей должников наглядно демонстрирует зависимость возврата кредита от цели кредита>

research['total_income_category'] = research['total_income'].apply(total_income_count)
total_income_dict = research[['total_income','total_income_category']]
total_income_dict['total_income_category'].value_counts()


уровень дохода выше среднего    5825
средний уровень дохода          5369
низкий уровень дохода           5368
высокий уровень дохода          4909
Name: total_income_category, dtype: int64

In [43]:
purpose_debt = research.pivot_table(index = 'category', columns = 'debt', values = 'dob_years', aggfunc = 'count')
purpose_debt.columns = ['Нет долга', 'Есть долг']
purpose_debt['Доля должников'] = (purpose_debt['Есть долг']
                                        /(purpose_debt['Есть долг'] 
                                          + purpose_debt['Нет долга'])).map('{:.1%}'.format)
purpose_debt.head()

,Нет долга,Есть долг,Доля должников
category,,,
Займ на автомобиль,3905,403,9.4%
Займ на недвижимость,10032,782,7.2%
Займ на образование,3644,370,9.2%
Займ на свадьбу,2149,186,8.0%


### Выводы

Да, цель кредита влияет на его возврат клиентом в срок.

1. Наиболее рискованными для банка целями кредитов являются займ на автомобиль и займ на образование. Возможно, это связано с тем, что кредиты на данные цели легче получить и суммы кредита ниже, чем на покупку недвижимости. 
2. Наименее рискованной целью кредита для банка является займ на недвижимость. Покупка квартиры - более серьезный шаг и кредиты на него составляют большие суммы, чем на покупку автомобиля или образование. Возможно, такое низкое значение долга является следствием того, что для кредита на покупку квартиры клиент проходит более сложный процесс получения кредита.
3. Наиболее близким к среднему значению в 8,45% является займ на свадьбу (8,0%). Этих займов меньше всего и риски для банка - ниже среднего значения по запросам на кредит.

### Шаг 4. Общий вывод <a id="total"></a>

Рискованнее выдавать кредиты тем клиентам, у которых есть дети.

Наиболее подвержены накоплению долгов неженатые/незамужные клиенты - почти 10% из них имеют долги. Также при выдаче кредита стоит обратить внимание на тех клиентов, которые состоят в гражданском браке - они недалеко ушли от неженатых/незамужных (9,5% долгов).

Больше всего должников среди клиентов со средним уровнем дохода и уровнем дохода выше среднего - (8,8% и 8,5%).

Наиболее рискованными для банка целями кредитов являются займ на автомобиль и займ на образование.